# **MediAssist: Your Generative AI Healthcare Companion**
#### Team Name: MedAssist Masters

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/haebin827/genai-finalProject/blob/main/HealthcareChatbotFile.ipynb)


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

%cd gdrive/My Drive

Current working directory: G:\내 드라이브


In [2]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j tiktoken yfiles_jupyter_graphs streamlit
%pip install ctransformers
!npm install localtunnel
!npm uninstall localtunnel
!npm install localtunnel

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


changed 1 package, and audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write



removed 22 packages, and audited 1 package in 3s

found 0 vulnerabilities


npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR UNKNOWN: unknown error, write
npm WARN tar TAR_ENTRY_ERROR EBADF: bad file descriptor, write
npm WARN tar TAR_ENTRY_ERROR EBADF: bad file descrip


added 22 packages, and audited 23 packages in 5s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [ ]:
# %%writefile main.py

from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough


try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

C:\Users\kkk\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["NEO4J_URI"] = ""
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = ""

In [ ]:
graph = Neo4jGraph()

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(["https://www.mayoclinic.org/diseases-conditions",
                        "https://www.mayoclinic.org/tests-procedures",
                        "https://www.mayoclinic.org/drugs-supplements",
                        "https://www.mayoclinic.org/healthy-lifestyle",
                        "https://www.mayoclinic.org/diseases-conditions/fever/symptoms-causes/syc-20352759",
                        "https://www.mayoclinic.org/first-aid/first-aid-fever/basics/art-20056685",
                        "https://www.mayoclinic.org/diseases-conditions/fever/in-depth/fever/art-20050997",
                        "https://www.mayoclinic.org/diseases-conditions/fever/diagnosis-treatment/drc-20352764",
                        "https://www.mayoclinic.org/symptoms/headache/basics/definition/sym-20050800",
                        "https://www.mayoclinic.org/diseases-conditions/tension-headache/in-depth/headaches/art-20046707",
                        "https://www.mayoclinic.org/departments-centers/headache-subspecialty-group/overview/ovc-20443693",
                        "https://www.mayoclinic.org/diseases-conditions/cluster-headache/symptoms-causes/syc-20352080",
                        "https://www.mayoclinic.org/diseases-conditions/tension-headache/symptoms-causes/syc-20353977",
                        "https://www.mayoclinic.org/diseases-conditions/headaches-in-children/symptoms-causes/syc-20352099",
                        "https://www.mayoclinic.org/diseases-conditions/migraine-headache/in-depth/migraines/art-20047242",
                        "https://www.mayoclinic.org/diseases-conditions/tension-headache/in-depth/headaches/art-20047631",
                        "https://www.mayoclinic.org/diseases-conditions/tension-headache/diagnosis-treatment/drc-20353982",
                        "https://www.mayoclinic.org/diseases-conditions/migraine-headache/diagnosis-treatment/drc-20360207",
                        "https://newsnetwork.mayoclinic.org/discussion/6-tips-for-headache-relief/",
                        "https://www.mayoclinic.org/symptoms/fatigue/basics/causes/sym-20050894",
                        "https://www.mayoclinic.org/diseases-conditions/chronic-fatigue-syndrome/symptoms-causes/syc-20360490",
                        "https://www.mayoclinic.org/diseases-conditions/cancer/in-depth/cancer-fatigue/art-20047709",
                        "https://www.mayoclinic.org/diseases-conditions/chronic-fatigue-syndrome/diagnosis-treatment/drc-20360510",
                        "https://www.mayoclinic.org/symptoms/abdominal-pain/basics/causes/sym-20050728",
                        "https://www.mayoclinic.org/symptoms/abdominal-pain/basics/definition/sym-20050728?redate=28032023&p=1",
                        "https://www.mayoclinic.org/symptom-checker/abdominal-pain-in-adults-adult/related-factors/itt-20009075",
                        "https://www.mayoclinic.org/diseases-conditions/irritable-bowel-syndrome/symptoms-causes/syc-20360016",
                        "https://www.mayoclinic.org/diseases-conditions/contact-dermatitis/symptoms-causes/syc-20352742",
                        "https://www.mayoclinic.org/diseases-conditions/poison-ivy/diagnosis-treatment/drc-20376490",
                        "https://www.mayoclinic.org/diseases-conditions/poison-ivy/symptoms-causes/syc-20376485",
                        "https://www.mayoclinic.org/diseases-conditions/heat-rash/symptoms-causes/syc-20373276",
                        "https://www.mayoclinic.org/healthy-lifestyle/infant-and-toddler-health/in-depth/baby-rashes/art-20546833",
                        "https://www.mayoclinic.org/symptoms/breast-rash/basics/causes/sym-20050817",
                        "https://www.mayoclinic.org/symptoms/cough/basics/definition/sym-20050846",
                        "https://www.mayoclinic.org/diseases-conditions/chronic-cough/symptoms-causes/syc-20351575",
                        "https://www.mayoclinic.org/diseases-conditions/chronic-cough/diagnosis-treatment/drc-20351580",
                        "https://www.mayoclinic.org/symptom-checker/cough-in-children-child/related-factors/itt-20009075",
                        "https://www.mayoclinic.org/symptoms/cough/basics/causes/sym-20050846?=lt%3B%2Fref%2Cgt&p=1",
                        "https://www.mayoclinic.org/symptom-checker/cough-in-adults-adult/related-factors/itt-20009075",
                        "https://www.mayoclinic.org/symptoms/cough/basics/when-to-see-doctor/sym-20050846",
"https://www.mayoclinic.org/departments-centers/cough-clinic/overview/ovc-20399029",
"https://www.mayoclinic.org/diseases-conditions/teen-depression/symptoms-causes/syc-20350985",
"https://www.mayoclinic.org/diseases-conditions/depression/symptoms-causes/syc-20356007",
"https://www.mayoclinic.org/diseases-conditions/seasonal-affective-disorder/symptoms-causes/syc-20364651",
"https://www.mayoclinic.org/diseases-conditions/mood-disorders/symptoms-causes/syc-20365057",
"https://www.mayoclinic.org/diseases-conditions/persistent-depressive-disorder/symptoms-causes/syc-20350929",
"https://www.mayoclinic.org/diseases-conditions/depression/in-depth/depression/art-20047725",
"https://www.mayoclinic.org/diseases-conditions/depression/expert-answers/pain-and-depression/faq-20057823",
"https://www.mayoclinic.org/diseases-conditions/mental-illness/symptoms-causes/syc-20374968",
"https://www.mayoclinic.org/diseases-conditions/depression/diagnosis-treatment/drc-20356013",
"https://www.mayoclinic.org/healthy-lifestyle/pregnancy-week-by-week/in-depth/depression-during-pregnancy/art-20237875",
"https://www.mayoclinic.org/symptoms/abdominal-pain/basics/when-to-see-doctor/sym-20050728",
"https://www.mayoclinic.org/symptoms/abdominal-pain/basics/when-to-see-doctor/sym-20050728?=re&p=1",
"https://www.mayoclinic.org/symptoms/diarrhea/basics/when-to-see-doctor/sym-20050926",
"https://www.mayoclinic.org/symptoms/abdominal-pain/basics/definition/sym-20050728",
"https://www.mayoclinic.org/diseases-conditions/gastritis/symptoms-causes/syc-20355807",
"https://www.mayoclinic.org/symptoms/nausea/basics/when-to-see-doctor/sym-20050736",
"https://www.mayoclinic.org/diseases-conditions/indigestion/symptoms-causes/syc-20352211",
"https://www.mayoclinic.org/diseases-conditions/gas-and-gas-pains/symptoms-causes/syc-20372709",
"https://www.mayoclinic.org/symptoms/abdominal-pain/basics/definition/sym-20050728?redate=28032023&p=1",
"https://www.mayoclinic.org/diseases-conditions/self-injury/diagnosis-treatment/drc-20350956",
"https://newsnetwork.mayoclinic.org/discussion/mayo-clinic-minute-how-to-self-care-at-home-when-you-have-covid-19/",
"https://newsnetwork.mayoclinic.org/discussion/self-care-tips-to-manage-mental-health-and-wellness/",
"https://www.mayoclinic.org/diseases-conditions/coronavirus/in-depth/mental-health-covid-19/art-20482731",
"https://www.mayoclinic.org/diseases-conditions/coronavirus/in-depth/treating-covid-19-at-home/art-20483273",
"https://newsnetwork.mayoclinic.org/discussion/home-remedies-self-care-approaches-to-treating-pain/",
"https://newsnetwork.mayoclinic.org/discussion/expert-alert-back-and-neck-pain-treatments-cover-self-care-to-surgery-prevention-is-key"
])
loader.requests_kwargs = {'verify':False}
data = loader.load()
# print(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\kkk\AppData\Roaming\Python\Python312\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.mayoclinic.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\kkk\AppData\Roaming\Python\Python312\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.mayoclinic.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\kkk\AppData\Roaming\Python\Python312\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.mayoclinic.org'. Adding certificate verification is strongly advised. See: h

In [6]:
len(data[0].page_content)
data[0].page_content[0:100]

'Medical Diseases & Conditions - Mayo Clinic\n\nSkip to contentCare at \n Mayo Clinic Patient-Centered C'

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(data)

print(documents[0])
print(documents[1])
print(documents[2])

page_content='Medical Diseases & Conditions - Mayo Clinic

Skip to contentCare at 
 Mayo Clinic Patient-Centered Care About Mayo Clinic Request Appointment Find a Doctor Locations Clinical Trials Connect to Support Groups Patient & Visitor Guide Billing & Insurance Departments & Centers International Services Contact UsPatient & Visitor Guide Health 
 Library Diseases & Conditions Symptoms Tests & Procedures Drugs & Supplements Healthy Lifestyle Books & SubscriptionsDiseases & Conditions For Medical 
 Professionals Medical Professional Resources Refer a Patient Continuing Medical Education AskMayoExpert Mayo Clinic Laboratories Video Center Journals & Publications Mayo Clinic Alumni AssociationContinuing Medical Education Research & Education 
 at Mayo ClinicResearch Research at Mayo Clinic Research Faculty Laboratories Core Facilities Centers & Programs Departments & Divisions Clinical Trials Institutional Review Board Postdoctoral Fellowships Training Grant ProgramsEducation Mayo Cli

In [ ]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125") # gpt-4-0125-preview occasionally has issues
llm_transformer = LLMGraphTransformer(llm=llm)

# Following needs to be used only first time to load the data. It took 3m 44s
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

print (documents[5])
print(graph_documents[5])

page_content='
Request an Appointment




Clinical trials at Mayo Clinic

Mayo Clinic conducts thousands of clinical trials and research studies each year.
Find active clinical trials






Featured topics






 
Cancer
Mayo Clinic cancer specialists use innovative therapies to treat even the rarest of cancers while helping you maintain your quality of life.
Chemotherapy - CancerChemotherapyMinimally invasive surgery - CancerMinimally invasive surgeryRadiation therapy - CancerRadiation therapyProton Beam Therapy Program - CancerProton Beam Therapy ProgramLearn more about cancer care at Mayo Clinic - CancerLearn more about cancer care at Mayo Clinic




 
Cardiovascular Diseases and Surgery
Mayo Clinic heart specialists have extensive experience in treating heart disease, and perform more than 10,000 cardiac procedures and 4,000 surgeries each year, including complex and minimally invasive surgical procedures.
Cardiovascular Surgery - Cardiovascular Diseases and SurgeryCardiovascular S

In [ ]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: MENTIONS)} {position: line: 1, column: 15, offset: 14} for query: 'MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50'


GraphWidget(layout=Layout(height='500px', width='100%'))

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
# Retriever

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the courses, requirements, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting course, professors, requirements, project entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

NameError: name 'graph' is not defined

In [ ]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [ ]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [ ]:
# Answer the question based only on the following context
template = """Answer the query:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Fever-related questions
chain.invoke({"question": "What are the common symptoms of fever?"})

Search query: What is the minimum score of Duolingo?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

'Structured data refers to data that is organized in a predefined format, such as databases or spreadsheets. Unstructured data refers to data that does not have a specific format, such as text documents or social media posts.'

In [ ]:
# True Positive
chain.invoke({"question": "Is a body temperature of 100.4°F considered a fever?"})

Search query: Is the minimum score for Dulingo 110 ?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

'Structured data refers to data that is organized in a predefined format, such as databases or spreadsheets. Unstructured data, on the other hand, refers to data that does not have a specific format or organization, such as text documents or social media posts. \n\nRegarding the query, the minimum score for Duolingo is not 110.'

In [ ]:
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger("neo4j.notifications").setLevel(logging.ERROR)

In [ ]:
# True Negative
chain.invoke({"question": "Is a body temperature of 98.6°F not considered a fever?"})

Search query: Is a  score of 100 not sufficient for Dulingo ?


'Structured data: A score of 100 is not sufficient for Duolingo.\n\nUnstructured data: It depends on the specific requirements of the Duolingo program or course.'

In [ ]:
# False Negative
chain.invoke({"question": "Is a body temperature of 102°F considered normal?"})

Search query: Is a  score of 115 not sufficient for Dulingo ?


'Structured data: A score of 115 is not sufficient for Duolingo.\n\nUnstructured data: The score of 115 is not enough for Duolingo.'

In [ ]:
# False Positives
chain.invoke({"question": "Is a body temperature of 99°F considered a high fever?"})

Search query: Is a  score of 105 sufficient for Dulingo ?


'Structured data: A score of 105 is sufficient for Duolingo.\n\nUnstructured data: Duolingo accepts a score of 105 as sufficient.'

In [ ]:
# Diabetes-related questions
chain.invoke({"question": "What are the common symptoms of diabetes?"})
chain.invoke({"question": "Does a fasting blood sugar level of 126 indicate diabetes?"})
chain.invoke({"question": "What are the long-term effects of untreated diabetes?"})

Search query: Who is the Advisor for MSACS program?


'Structured data: The Advisor for the MSACS program is Dr. John Smith.\n\nUnstructured data: Information about the Advisor for the MSACS program is not readily available in a structured format.'

In [ ]:
# Cancer-related questions
chain.invoke({"question": "What are the early signs of lung cancer?"})
chain.invoke({"question": "Can breast cancer be detected through regular mammograms?"})
chain.invoke({"question": "What are the common treatment options for skin cancer?"})

Search query: Is fire hot tell me T or F?


'True, fire is hot.'

In [ ]:
%%writefile main.py

from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough


try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

os.environ["OPENAI_API_KEY"] = ""
os.environ["NEO4J_URI"] = ""
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = ""

_search_query = RunnableLambda(lambda x : x["question"])


graph = Neo4jGraph()

llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125") # gpt-4-0125-preview occasionally has issues
llm_transformer = LLMGraphTransformer(llm=llm)

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)


graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the courses, requirements, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting course, professors, requirements, project entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f""" Structured data: {structured_data} Unstructured data: {"#Document ". join(unstructured_data)}  """
    return final_data


def answerquery(question: str):
  template = """Answer the question based only on the following context:
  {context}

  Question: {question}
  Use natural language and be concise.
  Answer:"""
  prompt = ChatPromptTemplate.from_template(template)

  chain = (
      RunnableParallel(
          {
              "context": _search_query | retriever,
              "question": RunnablePassthrough(),
          }
      )
      | prompt
      | llm
      | StrOutputParser()
  )

  return chain.invoke(({"question": question}))


Overwriting main.py


In [ ]:
%%writefile app.py

import streamlit as st
from main import answerquery

st.title("MediAssist QA Chatbot")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.text_input("What is up?"):
    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    response = answerquery(prompt)
    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        st.markdown(response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

Streamlit 서버가 백그라운드에서 실행 중입니다.


In [ ]:
!npx localtunnel --port 8501 & curl icanhazip.com

In [ ]:
!npm install localtunnel


up to date, audited 23 packages in 678ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
